# Assignment 9

### Shovan Biswas, Ben Horvath, Randy Thompson

### Data Set Information

Source: http://archive.ics.uci.edu/ml/datasets/Spambase

_Our collection of spam e-mails came from our postmaster and individuals who had filed spam. Our collection of non-spam e-mails came from filed work and personal e-mails, and hence the word 'george' and the area code '650' are indicators of non-spam. These are useful when constructing a personalized spam filter. One would either have to blind such non-spam indicators or get a very wide collection of non-spam to generate a general purpose spam filter._

From source website, we donwloaded the files `spambase.names` and `spambase.data`. The latter contains the actual data and the former contains the column names and directions to construct the file. Following the directions, we created spambase.csv.    

### Import and read    

In [123]:
import nltk
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import ensemble
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn.metrics as sm

import seaborn as sn

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import matplotlib.pyplot as plt

In [124]:
spam_data = pd.read_csv("spambase.csv")

### Basic data exploration   

The dataset contains 4601 rows, where each row corresponds to an e-mail. Each e-mail is described by fifty-seven features, and annotated with a class name (spam/not spam). The variables contain the porportion of the words in the e-mail that are a particular token or character, e.g., exclamation mark, the word 'address.'

In [125]:
spam_data.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spamclass
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


In [126]:
print("Content of this dataset: " + str(spam_data.shape[0]) + " rows and " + str(spam_data.shape[1]) + " columns")

Content of this dataset: 4601 rows and 58 columns


#### Column wise statistics of data    

In [127]:
spam_data.describe()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spamclass
count,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,...,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000
mean,0.104553,0.213015,0.280656,0.065425,0.312223,0.095901,0.114208,0.105295,0.090067,0.239413,...,0.038575,0.139030,0.016976,0.269071,0.075811,0.044238,5.191515,52.172789,283.289285,0.394045
std,0.305358,1.290575,0.504143,1.395151,0.672513,0.273824,0.391441,0.401071,0.278616,0.644755,...,0.243471,0.270355,0.109394,0.815672,0.245882,0.429342,31.729449,194.891310,606.347851,0.488698
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.588000,6.000000,35.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.065000,0.000000,0.000000,0.000000,0.000000,2.276000,15.000000,95.000000,0.000000
75%,0.000000,0.000000,0.420000,0.000000,0.380000,0.000000,0.000000,0.000000,0.000000,0.160000,...,0.000000,0.188000,0.000000,0.315000,0.052000,0.000000,3.706000,43.000000,266.000000,1.000000
max,4.540000,14.280000,5.100000,42.810000,10.000000,5.880000,7.270000,11.110000,5.260000,18.180000,...,4.385000,9.752000,4.081000,32.478000,6.003000,19.829000,1102.500000,9989.000000,15841.000000,1.000000


#### Spam / non-spam count     

The last column spamclass determines whether a data is spam or otherwise. If spamclass is 1, it's a spam and if it's 0, it's not a spam. (Ref: attribute descriptions in spambase.names).       

In [128]:
spam_column = len(spam_data[spam_data.spamclass == 1])
notspam_column = len(spam_data[spam_data.spamclass == 0])

print("Spam data count: %d" %spam_column)
print("Non spam data count: %d" %notspam_column)

Spam data count: 1813
Non spam data count: 2788


#### Check for nulls 

Forunately for us, there are no nulls in the dataset:

In [129]:
spam_data[spam_data.isnull().any(axis = 1)]

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spamclass


### Data Preparation

We'll split the data into train, validation, and test sets, with 70 percent in train.         

In [130]:
train_percentage = 0.7
val_percentage = 0.15

spam_data_rows = len(spam_data)
train_rows = int(spam_data_rows * train_percentage)
val_rows = int(spam_data_rows * val_percentage)
test_rows = spam_data_rows - train_rows - val_rows

In [131]:
print("Training rows (70 prc of total): %d" %train_rows)
print("Validation rows (15 prc of total): %d" %val_rows)
print("Testing rows (15 prc of total): %d" %test_rows)
print("Total: %d" %(train_rows + val_rows + test_rows))

Training rows (70 prc of total): 3220
Validation rows (15 prc of total): 690
Testing rows (15 prc of total): 691
Total: 4601


In [132]:
train_set, test_set = train_test_split(spam_data, test_size = test_rows, random_state = 8)
train_set, val_set = train_test_split(train_set, test_size = val_rows, random_state = 88)

In [133]:
print("Training set: %d" %len(train_set))
print("Validation set: %d" %len(val_set))
print("Testing set: %d" %len(test_set))
print("Total: %d" %(len(train_set) + len(val_set) + len(test_set)))

Training set: 3220
Validation set: 690
Testing set: 691
Total: 4601


### Confusion Matrix

The following function (just a function, no action is happening here) computes Confusion Matrix, which shows True Positives (TP), True Negatives (TN), False Positives (FP) and False Negatives (FN), from which the performance of an algorithm can be determined.    

In [134]:
def func_confusion_matrix(y_true, y_pred):
    cm = sm.confusion_matrix(y_true, y_pred, labels = [1, 0])
    print("TP: %d" %cm[0,0])
    print("FP: %d" %cm[1,0])
    print("TN: %d" %cm[1,1])
    print("FN: %d" %cm[0,1])
    print(sm.classification_report(y_true, y_pred, labels = [1,0], target_names = ["Spam", "Not spam"]))  
    return cm

### Data Analysis

We'll classify the data in three possible machine learning methods, namely Decision Tree, Random Forest and Support Vector Machines.      

#### Decision Tree

First, we'll classify using decision tree, where each node represents a portion of the data, whether it's spam or not.    

__We will use 10-fold cross validation to keep the learning algorithm from overfitting, to provide the best estimate of classifier performance possible, as well as hyperparameter tuning.__

#### Classification and model statistics for training set

Prepare the training data for input to the algorith:

In [135]:
train_class = train_set['spamclass']
train_vars = train_set.drop(labels = 'spamclass', axis = 1)

Next, set the infastructure for hyperparameter tuning on the train set:

In [136]:
from sklearn.metrics import recall_score, make_scorer

recall_scorer = make_scorer(recall_score)

param_grid = {'criterion': ['gini', 'entropy'],
              'max_depth': [2, 5, 10, 15, 25, 30],
              'min_samples_split': [10, 20, 50, 100]}

dt = tree.DecisionTreeClassifier()

# Instantiate grid search and tell it to maximize recall
dt_cv = GridSearchCV(dt, param_grid, cv=10, scoring=recall_scorer)

# Fit it to the data, and get info about the best model
dt_cv.fit(train_vars, train_class)
print(dt_cv.best_params_)

# Evaluate
dt_train = dt_cv.predict(train_vars)
func_confusion_matrix(train_class, dt_train)

{'criterion': 'entropy', 'max_depth': 25, 'min_samples_split': 10}
TP: 1230
FP: 17
TN: 1941
FN: 32
              precision    recall  f1-score   support

        Spam       0.99      0.97      0.98      1262
    Not spam       0.98      0.99      0.99      1958

   micro avg       0.98      0.98      0.98      3220
   macro avg       0.99      0.98      0.98      3220
weighted avg       0.98      0.98      0.98      3220



array([[1230,   32],
       [  17, 1941]])

The output above is very informative.

First, the optimal parameters of the decision tree are output. Second, is the classification matrix. And third shows detailed performance information.

__Particularly important to us is recall, which are optimizing for.__

#### Classification and model statistics for test set

The performance on the test set below is comparable to the cross-validated performance on the train set -- although the model is perhaps slightly overfitted.

_Results:_ We can say that the trained decision tree correctly identifies spam as spam about 91 percent of the time (precision), and correctly identifies non-spam as non-span 88 percent of th etime (recall).

In [137]:
test_class = test_set['spamclass']
test_vars = test_set.drop(labels='spamclass', axis = 1)

# Evaluate
dt_test = dt_cv.predict(test_vars)
func_confusion_matrix(test_class, dt_test)

TP: 244
FP: 29
TN: 383
FN: 35
              precision    recall  f1-score   support

        Spam       0.89      0.87      0.88       279
    Not spam       0.92      0.93      0.92       412

   micro avg       0.91      0.91      0.91       691
   macro avg       0.91      0.90      0.90       691
weighted avg       0.91      0.91      0.91       691



array([[244,  35],
       [ 29, 383]])

#### Determine feature importance

We can also go 'under the hood' of the model, to see what it computed are the most important variables. Note that the measure of importance, `imp`, is standarded to be between zero and one.

We see the frequency of the exclamation mark character (!) is by far the most indicative of a spam e-mail. The dollar sign character ($) as well as the word 'remove' form a second set of importance features. Then there appears to be a 'long tail' of less predictive words.

In [138]:
def func_arrange_feature_by_importance(fit, vars):
    df = {'variable': pd.Series(vars.columns.values), 'imp': pd.Series(fit.feature_importances_)}
    return pd.DataFrame(df, columns=['variable','imp']).sort_values(['imp'], ascending=0).head(10)

# use best_estimator_ to access the best performaing cross validated model
func_arrange_feature_by_importance(dt_cv.best_estimator_, test_vars)

,variable,imp
51,char_freq_!,0.306203
52,char_freq_$,0.136087
6,word_freq_remove,0.130127
54,capital_run_length_average,0.056622
55,capital_run_length_longest,0.050549
26,word_freq_george,0.033713
56,capital_run_length_total,0.027828
4,word_freq_our,0.027438
15,word_freq_free,0.023608
24,word_freq_hp,0.022861


#### Random Forest

Random Forest algorithm decreases the variance of a single decision tree. While the latter are implemented on an entire dataset, using all features (variables) of interest, a Random Forest randomly selects observations (i.e. rows) and specific features to build multiple Decision Trees, the computing an average measure.

#### Classification and model statistics for training set

In [139]:
#  to speed up training, plug in results from above:
#{'criterion': 'entropy', 'max_depth': 30, 'min_samples_split': 10}

param_grid = {'criterion': ['entropy'],
              'max_depth': [30],
              'min_samples_split': [10],
              'n_estimators': [500, 600, 700]}

forest = ensemble.RandomForestClassifier()

# Instantiate grid search and tell it to maximize recall
rf_cv = GridSearchCV(forest, param_grid, cv=10, scoring=recall_scorer)

# Fit it to the data, and get info about the best model
rf_cv.fit(train_vars, train_class)
print(rf_cv.best_params_)

# Evaluate
rf_train = rf_cv.predict(train_vars)
func_confusion_matrix(train_class, rf_train)

{'criterion': 'entropy', 'max_depth': 30, 'min_samples_split': 10, 'n_estimators': 600}
TP: 1235
FP: 11
TN: 1947
FN: 27
              precision    recall  f1-score   support

        Spam       0.99      0.98      0.98      1262
    Not spam       0.99      0.99      0.99      1958

   micro avg       0.99      0.99      0.99      3220
   macro avg       0.99      0.99      0.99      3220
weighted avg       0.99      0.99      0.99      3220



array([[1235,   27],
       [  11, 1947]])

The best performing Random Forest uses 600 trees.

#### Classification and model statistics for test set

The below shows how well this model performs on the test set. It appears to perform better than a decision tree alone. Only 33 observations of over 4000 are misclassified!

96 percent of spam e-mails are correctly classified as spam e-mails, and 92 percent of non-spam are correctly classified as non-spam.

In [140]:
rf_test = rf_cv.predict(test_vars)
func_confusion_matrix(test_class, rf_test)

TP: 258
FP: 12
TN: 400
FN: 21
              precision    recall  f1-score   support

        Spam       0.96      0.92      0.94       279
    Not spam       0.95      0.97      0.96       412

   micro avg       0.95      0.95      0.95       691
   macro avg       0.95      0.95      0.95       691
weighted avg       0.95      0.95      0.95       691



array([[258,  21],
       [ 12, 400]])

Again, we can determine the feature importance according to the Random Forest. Like the individual decision trees, the exclamation and dollars signs are the most importance.

In [141]:
func_arrange_feature_by_importance(rf_cv.best_estimator_, test_vars)

,variable,imp
51,char_freq_!,0.119229
52,char_freq_$,0.086826
6,word_freq_remove,0.083747
54,capital_run_length_average,0.062230
15,word_freq_free,0.056850
55,capital_run_length_longest,0.055157
20,word_freq_your,0.055032
24,word_freq_hp,0.053533
56,capital_run_length_total,0.043057
23,word_freq_money,0.034617


#### Support Vector Machines

Support Vector Machine (SVM) is a machine learning algorithms that is used to separate sets of points in a hyperspace, by a hyperplain. Train the algorithm:

In [142]:
from sklearn import svm

param_grid = {'C': [0.1, 1, 10, 100],
              'gamma': [1, 0.1, 0.01, 0.001]}

# Instantiate grid search and tell it to maximize recall
svm_cv = GridSearchCV(svm.SVC(), param_grid, cv=5, scoring=recall_scorer)

# Fit it to the data, and get info about the best model
svm_cv.fit(train_vars, train_class)
print(svm_cv.best_params_)

# Evaluate
svm_train = svm_cv.predict(train_vars)
func_confusion_matrix(train_class, svm_train)

{'C': 100, 'gamma': 0.001}
TP: 1206
FP: 38
TN: 1920
FN: 56
              precision    recall  f1-score   support

        Spam       0.97      0.96      0.96      1262
    Not spam       0.97      0.98      0.98      1958

   micro avg       0.97      0.97      0.97      3220
   macro avg       0.97      0.97      0.97      3220
weighted avg       0.97      0.97      0.97      3220



array([[1206,   56],
       [  38, 1920]])

Next, evalaute on the train set. We see that this particular SVM performs poorly compared to a decision tree or the random forest -- though this is probably because of sub-optimal hyper-tuning. 

Still, in the grand scheme of things this would be considered good performance. 86 percent of spam e-mails are correctly classified as spam, and 88 percent of non-spam e-mails are classified as non-spam.

In [143]:
svm_test = svm_cv.predict(test_vars)
func_confusion_matrix(test_class, svm_test)

TP: 245
FP: 39
TN: 373
FN: 34
              precision    recall  f1-score   support

        Spam       0.86      0.88      0.87       279
    Not spam       0.92      0.91      0.91       412

   micro avg       0.89      0.89      0.89       691
   macro avg       0.89      0.89      0.89       691
weighted avg       0.89      0.89      0.89       691



array([[245,  34],
       [ 39, 373]])